# Working Without the ibmpairs Python API Wrapper
The ibmpairs Python API Wrapper module handles a number of routine tasks that arise when communicating with Geospatial Analytics and processing query responses. While it is not necessary to use the module, doing so requires repeatedly writing some boiler plate code that essentially re-implements the wrappers functionality. Our minimal working example without using the ibmpairs Python API Wrapper takes the following form:

In [2]:
import os
import requests
import pandas as pd

EIS_USERNAME=os.environ.get('EIS_USERNAME')
EIS_APIKEY=os.environ.get('EIS_APIKEY')

AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'

# First we need to retrieve an access token using our api key
auth_response = requests.post(
    AUTH_PROVIDER_BASE + '/connect/token',
    headers={'Content-Type': 'application/x-www-form-urlencoded'},
    data=[
        ('client_id', 'ibm-pairs'),
        ('grant_type', 'apikey'),
        ('apikey', EIS_APIKEY)])
auth_obj = auth_response.json()

access_token = auth_obj.get('access_token')
refresh_token = auth_obj.get('refresh_token')

# Now we have the token we build the query JSON
query_json = {
     "layers" : [
         {"type" : "raster", "id" : "16100"}
     ],
     "spatial" : {"type" : "point",  "coordinates" : ["35.7", "139.7"]},
     "temporal" : {"intervals" : [
         {"start" : "2015-01-01T00:00:00Z", "end" : "2018-10-31T00:00:00Z"}
     ]}
 }

# And submit the query
api_response = requests.post(
     'https://pairs.res.ibm.com/v2/query',
     headers = {
         'Authorization':'Bearer {}'.format(access_token),
         'Content-Type': 'application/json'
     },
     json = query_json
 )
pairs_data = pd.DataFrame(api_response.json()['data'])
pairs_data['datetime'] = pd.to_datetime(pairs_data['timestamp'] * 1e6, errors = 'coerce')
pairs_data

,layerId,layerName,dataset,timestamp,longitude,latitude,value,datetime
0,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1421528400000,139.7,35.7,273.918212890625,2015-01-17 21:00:00
1,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1421539200000,139.7,35.7,276.215087890625,2015-01-18 00:00:00
2,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1421550000000,139.7,35.7,279.27020263671875,2015-01-18 03:00:00
3,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1421560800000,139.7,35.7,280.5292053222656,2015-01-18 06:00:00
4,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1421571600000,139.7,35.7,278.1947021484375,2015-01-18 09:00:00
...,...,...,...,...,...,...,...,...
11029,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1540900800000,139.7,35.7,292.9964904785156,2018-10-30 12:00:00
11030,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1540911600000,139.7,35.7,290.8324279785156,2018-10-30 15:00:00
11031,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1540922400000,139.7,35.7,289.5807189941406,2018-10-30 18:00:00
11032,16100,Ground temperature,16 day weather forecast (GFS) (latest predicti...,1540933200000,139.7,35.7,288.4697265625,2018-10-30 21:00:00


Instead of the ``ibmpairs`` modules that we import when using the ibmpairs Python API Wrapper we import the ``requests`` model that allows us to make API calls in Python. 

We use a POST request to convert our api key into a token. This was done for us by the authentication module in the first example. 

The query is defined in the ``query_json`` dictionary and the actual API request appears in the line:

```
api_response = requests.post(
     'https://pairs.res.ibm.com/v2/query',
     headers = {
         'Authorization':'Bearer {}'.format(access_token),
         'Content-Type': 'application/json'
     },
     json = query_json
 )
```
Here, the ``requests`` module is used to submit the query object to the Geospatial Analytics `/v2/query` endpoint. The JSON response from this request is use to create a Pandas data frame. 

We convert the timestamp returned by Geospatial Analytics from milliseconds epoch time to a human readable date. The API endpoint `POST/v2/query` is central to all Geospatial Analytics interactions. Indeed, the lion share of these tutorial pages concerns the various kinds of requests one can submit via `POST/v2/query`.

## Access Tokens


The ibmpairs Python API Wrapper handles using your API key to get an OAuth2 access token for you. 
```
auth_response = requests.post(
    AUTH_PROVIDER_BASE + '/connect/token',
    headers={'Content-Type': 'application/x-www-form-urlencoded'},
    data=[
        ('client_id', 'ibm-pairs'),
        ('grant_type', 'apikey'),
        ('apikey', EIS_APIKEY)])
auth_obj = auth_response.json()

access_token = auth_obj.get('access_token')
refresh_token = auth_obj.get('refresh_token')
```
The resulting OAuth ``access_token`` is a [JSON Web Token](https://en.wikipedia.org/wiki/JSON_Web_Token) that is set in the request ``Authorization`` header [Bearer](https://tools.ietf.org/html/rfc6750#section-2.1) scheme when sending request to Geospatial Analytics APIs.

## Refresh Tokens

Access token lifetime is not infinite. The Geospatial Analytics provider token expiry is 3600 seconds. After the expiry time of an access token has elapsed, Geospatial Analytics API calls issued with an expired access token will respond with a ``401`` *Unauthorized* HTTP status code.

Above, we get the ``refresh_token`` property.

```
refresh_token = auth_obj.get('refresh_token')
```

We use this property value to get a renewed access token.

```
 import requests
 AUTH_PROVIDER_BASE = 'https://auth-b2b-twc.ibm.com'

 refresh_token = auth_obj.get('refresh_token')

 auth_response = requests.post(
     AUTH_PROVIDER_BASE + '/connect/token',
     headers={'Content-Type': 'application/x-www-form-urlencoded'},
     data=[
         ('grant_type', 'refresh_token'),
         ('client_id', 'ibm-pairs'),
         ('refresh_token', refresh_token)])

 auth_obj = auth_response.json()
 access_token = auth_obj.get('access_token')
 print(access_token)
```

## Authentication Provider Details

To request access tokens outside of the ibmpairs Python API Wrapper you need to use the token request endpoint         ``https://auth-b2b-twc.ibm.com/connect/token``

### Access Token Payload

```
 {
   "grant_type":"apikey",
   "client_id":"ibm-pairs",
   "apikey":"<YOUR API KEY>"
 }
```

### Refresh Token Payload

The value for the ``refresh_token`` comes from the property of the same name in original access token response. 

```
 {
   "grant_type":"refresh_token",
   "client_id":"ibm-pairs",
   "refresh_token":"<refresh_token> from Token response"
 }
```

### Token Response

A response to a successful request to the API key bearer endpoint will produce a JSON object with the following structure.

```
 {
   "access_token": "<header>.<payload>.<signature>",
   "expires_in": 3600,
   "token_type": "Bearer",
   "refresh_token": "",
   "scope": ""
 }

```